In [ ]:
import tensorflow as tf
from tensorflow import keras as ks
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.style as stl
stl.use("ggplot")
import cv2 as cv
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
imageData = cv.capture()

In [ ]:
train_datagenerator = ImageDataGenerator(
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    rescale=1./255, # Nomalization of data. Chose 255 because we code each pixel by rgba
    rotation_range=40, # Rotation to gain generalization
    width_shift_range=0.2, 
    height_shift_range=0.2, 
    shear_range=.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest',
    validation_split=0.2)

# Generates batches for training (120 x 120 tiles)
train_generator = train_datagenerator.flow_from_dataframe(
    imageData, 
    x_col='path', 
    y_col='type',
    target_size=(120, 120), 
    color_mode='rgba', 
    class_mode='categorical', 
    batch_size=32, 
    shuffle=True, 
    seed=5201314,
    subset='training')

# Generates batches for testing (120 x 120 tiles)
validation_generator = train_datagenerator.flow_from_dataframe(
    imageData, 
    x_col='path',
    y_col='type',
    target_size=(120, 120),
    color_mode='rgba', 
    class_mode='categorical', 
    batch_size=4, 
    shuffle=True,
    seed=5201314, 
    subset='validation'
)

In [ ]:
# Model Initialization
model = ks.models.Sequential()

model.add(ks.layers.Dense(4, input_shape=(120, 120, 4)))

# Add first convolutional layer 
model.add(ks.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(ks.layers.MaxPooling2D(2, 2))

# Add second convolutional layer
model.add(ks.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(ks.layers.MaxPooling2D(2, 2))

# Add second convolutional layer
model.add(ks.layers.Conv2D(256, (3, 3), activation='relu'))
model.add(ks.layers.MaxPooling2D(2, 2))
model.add(ks.layers.Dropout(0.2))

# Flattens the imagesfrom high dimensions to low dimenions
model.add(ks.layers.Flatten())
        
model.add(ks.layers.Dense(1024, activation='relu'))

model.add(ks.layers.Dense(4, activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
history = model.fit(
    train_generator, 
    epochs=95
)